# GOAL: Text Generation using RNNs.
Given a sequence of characters, the task is to train a model which can predict the next character that occurs.

**Note:** This notebook is almost a direct replica of Tensorflow tutorial. Intended for practice.


In [1]:
# Importing essential libraries
import tensorflow as tf
import numpy as np
import os
import time

In [2]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


# Data Preparation

In [3]:
# Download the shakespere's text file
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')
print(path_to_file)

C:\Users\ichbi\.keras\datasets\shakespeare.txt


In [4]:
# Read the file as a buffered reader
text = open(path_to_file, "rb"); print(type(text))
# Read the text file through the buffered reader and decode into UTF-8 format for python to read it in a formatted manner.
text = text.read().decode(encoding="utf-8"); print(type(text));
# Display the length of characters
print(f"Length of text: {len(text)} characters.")

<class '_io.BufferedReader'>
<class 'str'>
Length of text: 1115394 characters.


In [5]:
# Print the first few lines of the shakespere's play
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [6]:
# Break all the words and store the unique characters.
## RESULT: VOCABULARY OF CHARACTERS
vocab = sorted(set(text))
print(vocab)
print(f"{len(vocab)} unique characters.")

['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
65 unique characters.


# Data Processing

## Tokenize the text

In [7]:
# Sample examples to test out if the tokeization is successful
example_texts = ["today I will be learning natural language processing", "okay thats it"]

# Divde the sentence into characters for tokenization
characters = tf.strings.unicode_split(example_texts, input_encoding="UTF-8")

In [8]:
# Lookup table to convert the characters to its corresponding numerical format
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)

# Finally, generate integer mapping for the characters from the examples to get a tokenized output
## RESULT: TOKENIZED RAGGED TENSOR, with each row containing integer mappings for corresponding characters in the sentences. 
ids = ids_from_chars(characters)
print(ids)

<tf.RaggedTensor [[59, 54, 43, 40, 64, 2, 22, 2, 62, 48, 51, 51, 2, 41, 44, 2, 51, 44, 40,
  57, 53, 48, 53, 46, 2, 53, 40, 59, 60, 57, 40, 51, 2, 51, 40, 53, 46, 60,
  40, 46, 44, 2, 55, 57, 54, 42, 44, 58, 58, 48, 53, 46]                   ,
 [54, 50, 40, 64, 2, 59, 47, 40, 59, 58, 2, 48, 59]]>


In [9]:
# To obtain a mapping from the encoded integer mapping back to string, create an invertible lookup table
chars_from_ids = tf.keras.layers.StringLookup(vocabulary = ids_from_chars.get_vocabulary(), mask_token = None, invert = True)

# Testing the inverse from vector to string and converting to words
## RESULTS: CHARACTERS converted from integers. HOPEFULLY ORIGINAL SENTENCES if inverse mapping worked
chars = chars_from_ids(ids)
print(tf.strings.reduce_join(chars, axis = -1).numpy())

# Write a function to later use it for joining the characters and reading the sentence
def text_from_ids(ids):
    return tf.strings.reduce_join(chars_from_ids(ids), axis = -1).numpy()

[b'today I will be learning natural language processing' b'okay thats it']


# Preparing the data to create training sequences for the model

We now divide the text into example sequences. Each input sequence should contain **sequence_length** number of characters. 

The corresponding target contains the same length of text, except shifted one character to the right.

Now, we break the chunk of text into seq_length+1 size.
    
    e.g., if **seq_length** = 4
          input_text = "hello"
          Then, sequence would be: "hell" and target sequence will be: "ello".

In [10]:
# So we essentially used the vocabulary we created using the shakespere's play by tokenizing them.
## and we later used the encoding (i.e., StringLookup) object to encode the orignal play's characters.
## OUTPUT: TOKENIZED SHAKESPEARES PLAY TEXT
all_ids = ids_from_chars(tf.strings.unicode_split(text, input_encoding="UTF-8"))
all_ids

# Creates a Tensorflow's dataset object through all the tensor slices created. (i.e., no of lists containing sentences)
## i.e., if two sentences were provided for encoding, two different lists are created. Hence, two internal 
## sets are created in dataset object, preserving the data structure. 
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

# Displaying the first 15 characters from the created TF dataset object
for ids in ids_dataset.take(15):
    print(chars_from_ids(ids).numpy().decode("UTF-8"))

F
i
r
s
t
 
C
i
t
i
z
e
n
:




In [11]:
# Sequence length for providing context to the model for learning the probabilities of next characters.
seq_length = 100
# add it as a batch size to output seq_len number of characters at once.
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

# Test the changes
for seq in sequences.take(1):
    print(chars_from_ids(seq))

# Merge the characters to form sentences
for seq in sequences.take(5):
    print(text_from_ids(seq))

tf.Tensor(
[b'F' b'i' b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':'
 b'\n' b'B' b'e' b'f' b'o' b'r' b'e' b' ' b'w' b'e' b' ' b'p' b'r' b'o'
 b'c' b'e' b'e' b'd' b' ' b'a' b'n' b'y' b' ' b'f' b'u' b'r' b't' b'h'
 b'e' b'r' b',' b' ' b'h' b'e' b'a' b'r' b' ' b'm' b'e' b' ' b's' b'p'
 b'e' b'a' b'k' b'.' b'\n' b'\n' b'A' b'l' b'l' b':' b'\n' b'S' b'p' b'e'
 b'a' b'k' b',' b' ' b's' b'p' b'e' b'a' b'k' b'.' b'\n' b'\n' b'F' b'i'
 b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':' b'\n' b'Y'
 b'o' b'u' b' '], shape=(101,), dtype=string)
b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
b'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
b"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
b"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be

In [12]:
# Function to split the sequences into its corresponding input and target sequences.
## OUTPUT: INPUT and TARGET both of length seq_len to be later fed into the model
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:seq_length+1]
    return input_text, target_text

# Testing
split_input_target(list("Tensorflow"))

# Mapping this function to the sequence generator to be used in TensorFlow RNN model
sequence_dataset = sequences.map(split_input_target)

for input_example, target_example in sequence_dataset.take(1):
    print("input", text_from_ids(input_example))
    print("Target", text_from_ids(target_example))

input b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target b'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


# Actual Training Logic

In [13]:
# Batch Size
BATCH_SIZE = 16

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).

BUFFER_SIZE = 10000

model_dataset = (
    sequence_dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE)
)

model_dataset

<PrefetchDataset element_spec=(TensorSpec(shape=(16, 100), dtype=tf.int64, name=None), TensorSpec(shape=(16, 100), dtype=tf.int64, name=None))>

In [14]:
# Length of the vocabulary in StringLookup Layer
vocab_size  = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [15]:
class RNNModel(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, rnn_units):
        super().__init__(self)
        self.embeddings = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(rnn_units,
                                       return_sequences=True,
                                       return_state = True)
        self.dense = tf.keras.layers.Dense(vocab_size)

    def call(self, inputs, states = None, return_state=False, training = False):
        x = inputs
        x = self.embeddings(x, training = training)
        if states is None:
            states = self.gru.get_initial_state(x)
        x, states = self.gru(x, initial_state=states, training = training)
        x = self.dense(x, training=training)

        if return_state:
            return x, states
        else:
            return x

In [16]:
model = RNNModel(
    vocab_size = vocab_size,
    embedding_dim = embedding_dim,
    rnn_units = rnn_units
)

# Try out the model without training

In [17]:
for input_example_batch, target_example_batch in model_dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")


(16, 100, 66) # (batch_size, sequence_length, vocab_size)


In [18]:
model.summary()

Model: "rnn_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  16896     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  67650     
                                                                 
Total params: 4,022,850
Trainable params: 4,022,850
Non-trainable params: 0
_________________________________________________________________


In [19]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [20]:
sampled_indices

array([43, 16, 15, 39, 41, 14, 21, 33, 15, 53, 12, 63, 40,  3, 64, 11, 42,
       10, 32,  7, 44, 15, 11,  8, 53, 41, 30, 62, 39,  2, 29, 50,  1, 39,
       35, 55, 14, 58, 47, 54, 39, 20, 37, 54,  2, 32, 34,  6,  4,  7, 65,
       35, 17, 42, 12, 17, 34, 40, 29, 13, 59, 59, 32, 39, 56, 42, 22,  1,
       13, 65, 12, 24,  4, 32, 18, 60, 35, 24, 40, 60, 47, 17, 57, 37, 34,
       32, 41,  8, 60, 50, 21, 41, 53, 60, 26, 63, 64,  0, 28, 48],
      dtype=int64)

In [21]:
print(text_from_ids(input_example_batch[0]))
print(text_from_ids(sampled_indices))

b"uch a name,\nWhose repetition will be dogg'd with curses;\nWhose chronicle thus writ: 'The man was nob"
b"dCBZbAHTBn;xa!y:c3S,eB:-nbQwZ Pk\nZVpAshoZGXo SU'$,zVDc;DUaP?ttSZqcI\n?z;K$SEuVKauhDrXUSb-ukHbnuMxy[UNK]Oi"


# Train the model

In [22]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [23]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (16, 100, 66)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.1889067, shape=(), dtype=float32)


In [24]:
model.compile(optimizer="adam", loss = loss)

# Directory where the checkpoints are saved
checkpoint_dir = "./training_checkpoints"
# Name of checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath = checkpoint_prefix,
    save_weights_only = True
)

In [26]:
##### EPOCHS = 20

history = model.fit(model_dataset, epochs = EPOCHS, callbacks = [checkpoint_callback])

Epoch 1/20
690/690 [==============================] - 12s 17ms/step - loss: 1.3332
Epoch 2/20
690/690 [==============================] - 12s 17ms/step - loss: 1.2648
Epoch 3/20
690/690 [==============================] - 12s 17ms/step - loss: 1.2030
Epoch 4/20
690/690 [==============================] - 12s 17ms/step - loss: 1.1435
Epoch 5/20
690/690 [==============================] - 12s 17ms/step - loss: 1.0830
Epoch 6/20
690/690 [==============================] - 12s 17ms/step - loss: 1.0226
Epoch 7/20
690/690 [==============================] - 12s 17ms/step - loss: 0.9669
Epoch 8/20
690/690 [==============================] - 13s 17ms/step - loss: 0.9167
Epoch 9/20
690/690 [==============================] - 13s 17ms/step - loss: 0.8756
Epoch 10/20
690/690 [==============================] - 13s 18ms/step - loss: 0.8438
Epoch 11/20
690/690 [==============================] - 13s 18ms/step - loss: 0.8202
Epoch 12/20
690/690 [==============================] - 13s 18ms/step - loss: 0.8051
E

In [40]:
# Class to run the model on 
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=0.5):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    #predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [41]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [75]:
start = time.time()
states = None
next_char = tf.constant(['Siv:'])
result = [next_char]

for n in range(10):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)


ROMEO:
A fellow  

________________________________________________________________________________

Run time: 0.12838125228881836
